In [1]:
import math
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
from contextlib import closing
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext


import hashlib


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
!pip install -q google-cloud-storage==1.43.0

In [3]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'project_ir313254724_322769175_title_final'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

In [4]:
def tokenize(text):
    def get_date_pattern():
      return r'((Jan(uary)?|Mar(ch)?|May|Jul(y)?|Aug(ust)?|Oct(ober)?|Dec(ember)?)\s([1-2][0-9]|[1-9]|[3][01]),\s([1-2][0-9]{3})|(Feb(ruary)?)\s([1-2][0-9]|[1-9]),\s([1-2][0-9]{3})|(Apr(il)?|Jun(e)?|Sep(tember)?|Nov(ember)?)\s([1-2][0-9]|[1-9]|30),\s([1-2][0-9]{3})|(([1-2][0-9]|[1-9]|3[01])\s(Jan(uary)?|Mar(ch)?|May|Jul(y)?|Aug(ust)?|Oct(ober)?|Dec(ember)?)\s[1-2][0-9]{3})|(([1-2][0-9]|[1-9])\s(Feb(uary)?)\s[1-2][0-9]{3})|(([1-2][0-9]|[1-9]|30)\s(Apr(il)?|Jun(e)?|Sep(tember)?|Nov(ember)?)\s[1-2][0-9]{3}))'
    def get_time_pattern():
      return r'(\b([1-9]|(1[0-2]))\.([0-5][0-9])(AM|PM)|(0[0-9]|1[0-2])[0-5][0-9](a\.m\.|p\.m\.)|\b([1-9]|1[0-9]|2[0-3]|00):[0-5][0-9]:[0-5][0-9]\b)'
    def get_percent_pattern():
      return r'((([1-9][0-9]{0,2})+(\,[0-9]{3})*|0)(\.\d+)?)%'
    def get_number_pattern():
      return r'(?<![\w\+\-\,\.])([+-]?[1-9]\d{0,2}((\,\d{3})*|\d*)|0)(\.\d+)?(?!(\.[^\s]|\,[^\s]|\w))'
    def get_word_pattern():
      return r"""[\#\@\w](['\-]?\w){2,24}"""

    RE_TOKENIZE = re.compile(rf"""
    (
        # dates
        |(?P<DATE>{get_date_pattern()})
        # time
        |(?P<TIME>{get_time_pattern()})
        # Percents
        |(?P<PERCENT>{get_percent_pattern()})
        # Numbers
        |(?P<NUMBER>{get_number_pattern()})
        # Words
        |(?P<WORD>{get_word_pattern()})
        )""",  re.MULTILINE | re.IGNORECASE | re.VERBOSE | re.UNICODE)
    return [token.group() for token in RE_TOKENIZE.finditer(text.lower()) if (token.group().strip() and token.group() not in stop_words)]

def word_count(text, id, stem=False):
    ''' Count the frequency of each word in text (tf) that is not included in
    all_stopwords and return entries that will go into our posting lists.
    Parameters:
    -----------
    text: str
    Text of one document
    id: int
    Document id
    Returns:
    --------
    List of tuples
    A list of (token, (doc_id, tf)) pairs
    for example: [("Anarchism", (12, 5)), ...]
    '''
    global_dict = {}
    all_tokens = tokenize(text)
    tokens = stemming(all_tokens, stem)
    for token in tokens:
      val = global_dict.get(token)
      if val is None:
        global_dict[token] = 1
      else:
        global_dict[token] += 1
    final = [(token, (id, freq)) for token, freq in global_dict.items()]
    return final

'''
sorting the posting list by doc_id. we will apply this function with map reduce
and it will be ended as dict of token: posting_list key-value.
'''
def reduce_word_counts(unsorted_pl):
    ''' Returns a sorted posting list by wiki_id.
    Parameters:
    -----------
    unsorted_pl: list of tuples
    A list of (wiki_id, tf) tuples
    Returns:
    --------
    list of tuples
    A sorted posting list.
    '''
    sorted_pl = sorted(unsorted_pl, key=lambda x: x[0])  # Sorting by wiki_id
    return sorted_pl

'''
idf added to this dict. this will be dict of {token: [df, idf]} and it will help to calculate the BM25.
'''
def calculate_df(postings, N):
    ''' Takes a posting list RDD and calculate the df for each token.
    Parameters:
    -----------
    postings: RDD
    An RDD where each element is a (token, posting_list) pair.
    Returns:
    --------
    RDD
    An RDD where each element is a (token, df) pair.
    '''
    return postings.map(lambda x: (x[0], (len(x[1]), math.log(N / len(x[1])))))

def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

def token2bucket_id(token):
    NUM_BUCKETS = 124
    return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings):
    ''' A function that partitions the posting lists into buckets, writes out
    all posting lists in a bucket to disk, and returns the posting locations for
    each bucket. Partitioning should be done through the use of token2bucket
    above. Writing to disk should use the function  write_a_posting_list, a
    static method implemented in inverted_index_colab.py under the InvertedIndex
    class.
    Parameters:
    -----------
    postings: RDD
    An RDD where each item is a (w, posting_list) pair.
    Returns:
    --------
    RDD
    An RDD where each item is a posting locations dictionary for a bucket. The
    posting locations maintain a list for each word of file locations and
    offsets its posting list was written to. See write_a_posting_list for
    more details.
    '''
    #create rdd of (bucket_id,(w, posting_list))
    bucket_postrdd = postings.map(lambda post_rdd: (token2bucket_id(post_rdd[0]),post_rdd)).groupByKey()
    final = bucket_postrdd.map(lambda buck: InvertedIndex.write_a_posting_list(buck,"title_posting/",bucket_name))
    return final

def stemming(tokens, stem=False):
    length_list = len(tokens)
    if stem:
      stemmer = PorterStemmer()
      index = 0
      while index < length_list:
          tokens[index] = stemmer.stem(tokens[index])
          index += 1
    return tokens

In [5]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs_title = parquetFile.select("title", "id").rdd

In [6]:
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [7]:
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())
from inverted_index_gcp import *

In [8]:
stop_words1 = set(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

stop_words = stop_words1.union(corpus_stopwords)

In [9]:
# Tokenize each document and count the number of tokens
doc_token_counts_title = doc_text_pairs_title.map(lambda x: (x[1], len(tokenize(x[0]))))

# Calculate the total number of documents and the total number of tokens
total_documents_title = doc_text_pairs_title.count()
total_tokens_title = doc_token_counts_title.map(lambda x: x[1]).sum()

# Calculate the average number of tokens per document
avg_tokens_per_doc_title = total_tokens_title / total_documents_title

# word counts map
flag_title = True
# model = Word2Vec(vector_size=300, window=10, min_count=5, workers=4)
word_counts_title = doc_text_pairs_title.flatMap(lambda x: word_count(x[0], x[1], flag_title))

postings_title = word_counts_title.groupByKey().mapValues(lambda x: reduce_word_counts(list(x)))
w2df_title = calculate_df(postings_title, total_documents_title)
w2df_dict_title = w2df_title.collectAsMap()

# partition posting lists and write out
_ = partition_postings_and_write(postings_title).collect()

In [10]:
# collect all posting lists locations into one super-set
super_posting_locs_title = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='title_posting'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_title[k].extend(v)

In [11]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 71.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.6 MB/s eta 0:00:00


In [12]:
import gensim.models
import gensim.downloader as api
model_glove = api.load("glove-wiki-gigaword-300")

[===-----------------------------------------------] 6.2% 23.4/376.1MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=======-------------------------------------------] 15.9% 59.9/376.1MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[==========----------------------------------------] 21.0% 78.9/376.1MB downloaded

In [13]:
# Create inverted index instance
inverted_title = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_title.posting_locs = super_posting_locs_title
# Add the token - df dictionary to the inverted index
inverted_title.df = w2df_dict_title
inverted_title.stem = flag_title
inverted_title.stop_words = stop_words
inverted_title.model = model_glove

'''
insert important values into InvertedIndex object.
'''
inverted_title.N = total_documents_title
inverted_title.avg_doclen = avg_tokens_per_doc_title
inverted_title.docs_len = doc_token_counts_title.collectAsMap() # Dict of doc_id: len_doc.

In [14]:
inverted_title.write_index('.', 'title_index')
index_src_title = "title_index.pkl"
index_dst_title = f'gs://{bucket_name}/title_posting/{index_src_title}'
!gsutil cp $index_src_title $index_dst_title
inverted_title.write_index('.', 'title_index')

Copying file://title_index.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\ [1 files][590.8 MiB/590.8 MiB]   29.5 MiB/s                                   
Operation completed over 1 objects/590.8 MiB.                                    
